# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [6]:
import requests
import re
from bs4 import BeautifulSoup

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [7]:
input_keyword = "設計思考"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

設計思考: %E8%A8%AD%E8%A8%88%E6%80%9D%E8%80%83
/wiki/%E8%A8%AD%E8%A8%88%E6%80%9D%E8%80%83


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [8]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


設計思考或設計思維（Design Thinking）是一個以人為本的解決問題方法論，透過從人的需求出發，為各種議題尋求創新解決方案，並創造更多的可能性。IDEO設計公司總裁提姆·布朗曾在《哈佛商業評論》定義：「設計思考是以人為本的設計精神與方法，考慮人的需求、行為，也考量科技或商業的可行性。」[1]

設計思考，與分析式思考（analytical thinking）相較之下，在「理性分析」層面是有很大不同的，設計思考是一種較為「感性分析」，並注重「了解」、「發想」、「構思」、「執行」的過程。目前多數教學都將設計思考過程，濃縮成五大步驟「同理心」、「需求定義」、「創意動腦」、「製作原型」、「實際測試」。[2]

設計思考是一個創意解難的過程，透過運用設計思考工具中的的元素，例如同理心、實驗等，以達致一個創新的解決方案。透過採用設計思考所作出的決定，是基於潛在客戶真正希望得到的事物，而非只基於歷史數據; 或是基於證據而非透過直覺而作出的帶有風險的決定。

以斯坦福大學普拉特納設計學院（英語：Hasso Plattner Institute of Design）（D-School）的設計思考訓練，提出一些在設計思考中具備的精神，如下所示：

設計思考的簡單流程可分於下述數個單元所組成。

Visio;Project;Paper Writing;Team Work and Concern Management



### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [9]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [IDEO] /wiki/IDEO
外部連結: [提姆·布朗] /wiki/%E6%8F%90%E5%A7%86%C2%B7%E5%B8%83%E6%9C%97
外部連結: [哈佛商業評論] /wiki/%E5%93%88%E4%BD%9B%E5%95%86%E6%A5%AD%E8%A9%95%E8%AB%96
外部連結: [斯坦福大學] /wiki/%E6%96%AF%E5%9D%A6%E7%A6%8F%E5%A4%A7%E5%AD%B8


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [14]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        '''
        
        Your code here
        
        '''
        title = html.find('title').text
        title = title.split('- 維基百科，自由的百科全書')[0]
        #print(title) 
        try:
            #創建WikiArticle檔案
            os.makedirs( './WikiArticle', exist_ok=True )
            for paragraph in content:
                #創建txt檔
                f =open("./WikiArticle/{}.txt".format(title),'a')
                f.write(paragraph.get_text())
                f.write('\n')
    
        except:
            print('發生錯誤！')


        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        '''
        
        Your code here
        
        '''
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       #網址
                    a_keyword = link_string.get_text() #名稱
                    external_link_dict[a_keyword]=a_link

                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = 2 
            
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  
                

### 執行前個步驟定義好的爬蟲主程式

In [15]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E8%A8%AD%E8%A8%88%E6%80%9D%E8%80%83 (設計思考)
發生錯誤！
遞迴層[1] - IDEO (/wiki/IDEO)


ConnectionError: HTTPSConnectionPool(host='zh.wikipedia.orgideo', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001BDE3BEAB38>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))